# What Dis

Consolidated crime data, curated from the FBI and city-data.com

# Imports and Definitions

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
abbreviations = {
    "Alabama"       : "AL",
    "Alaska"        : "AK",
    "Arizona"       : "AZ",
    "Arkansas"      : "AR",
    "California"    : "CA",
    "Colorado"      : "CO",
    "Connecticut"   : "CT",
    "Delaware"      : "DE",
    "Florida"       : "FL",
    "Georgia"       : "GA",
    "Hawaii"        : "HI",
    "Idaho"         : "ID",
    "Illinois"      : "IL",
    "Indiana"       : "IN",
    "Iowa"          : "IA",
    "Kansas"        : "KS",
    "Kentucky"      : "KY",
    "Louisiana"     : "LA",
    "Maine"         : "ME",
    "Maryland"      : "MD",
    "Massachusetts" : "MA",
    "Michigan"      : "MI",
    "Minnesota"     : "MN",
    "Mississippi"   : "MS",
    "Missouri"      : "MO",
    "Montana"       : "MT",
    "Nebraska"      : "NE",
    "Nevada"        : "NV",
    "New Hampshire" : "NH",
    "New Jersey"    : "NJ",
    "New Mexico"    : "NM",
    "New York"      : "NY",
    "North Carolina": "NC",
    "North Dakota"  : "ND",
    "Ohio"          : "OH",
    "Oklahoma"      : "OK",
    "Oregon"        : "OR",
    "Pennsylvania"  : "PA",
    "Rhode Island"  : "RI",
    "South Carolina": "SC",
    "South Dakota"  : "SD",
    "Tennessee"     : "TN",
    "Texas"         : "TX",
    "Utah"          : "UT",
    "Vermont"       : "VT",
    "Virginia"      : "VA",
    "Washington"    : "WA",
    "West Virginia" : "WV",
    "Wisconsin"     : "WI",
    "Wyoming"       : "WY",
}

In [3]:
places = {
    "Alabama": [
        "Birmingham",
        "Montgomery",],
    "Alaska": [
        "Anchorage",
        "Juneau",],
    "Arizona": [
        "Phoenix",
        "Tucson",],
    "Arkansas": [
        "Little Rock",
        "Fort Smith",],
    "California": [
        "San Diego",
        "Los Angeles",
        "Sacramento",],
    "Colorado": [
        "Denver",
        "Colorado Springs",
        "Aurora",],
    "Connecticut": [
        "Bridgeport",
        "Hartford",],
    "Delaware": [
        "Dover",
        "Wilmington",],
    "Florida": [
        "Jacksonville",
        "Miami",
        "Tallahassee",],
    "Georgia": [
        "Atlanta",
        "Savannah",],
    "Hawaii": [
        "Honolulu",
        "Kauai",
        "Maui",],
    "Idaho": [
        "Boise",
        "Meridian",],
    "Illinois": [
        "Chicago",
        "Springfield",],
    "Indiana": [
        "Fort Wayne",
        "Indianapolis",],
    "Iowa": [
        "Cedar Rapids",
        "Des Moines",],
    "Kansas": [
        "Topeka",
        "Wichita",],
    "Kentucky": [
        "Frankfort",
        "Louisville",],
    "Louisiana": [
        "Baton Rouge",
        "New Orleans",],
    "Maine": [
        "Augusta",
        "Portland",],
    "Maryland": [
        "Baltimore",
        "Annapolis",],
    "Massachusetts": [
        "Boston",
        "Worcester",],
    "Michigan": [
        "Detroit",
        "Lansing",],
    "Minnesota": [
        "Minneapolis",
        "Saint Paul",],
    "Mississippi": [
        "Jackson",
        "Gulfport",],
    "Missouri": [
        "Kansas City",
        "Jefferson City",],
    "Montana": [
        "Billings",
        "Helena",],
    "Nebraska": [
        "Lincoln",
        "Omaha",],
    "Nevada": [
        "Carson City",
        "Las Vegas",
        "Reno",],
    "New Hampshire": [
        "Concord",
        "Manchester",],
    "New Jersey": [
        "Newark",
        "Trenton",],
    "New Mexico": [
        "Albuquerque",
        "Santa Fe",],
    "New York": [
        "Albany",
        "New York City",],
    "North Carolina": [
        "Charlotte",
        "Raleigh",],
    "North Dakota": [
        "Bismarck",
        "Fargo",],
    "Ohio": [
        "Columbus",
        "Cleveland",],
    "Oklahoma": [
        "Oklahoma City",
        "Tulsa",],
    "Oregon": [
        "Portland",
        "Salem",],
    "Pennsylvania": [
        "Harrisburg",
        "Philadelphia",],
    "Rhode Island": [
        "Providence",
        "Warwick",],
    "South Carolina": [
        "Charleston",
        "Columbia",],
    "South Dakota": [
        "Pierre",
        "Sioux Falls",],
    "Tennessee": [
        "Nashville",
        "Memphis",],
    "Texas": [
        "Austin",
        "Houston",],
    "Utah": [
        "Salt Lake City",
        "West Valley City",],
    "Vermont": [
        "Burlington",
        "Montpelier",],
    "Virginia": [
        "Richmond",
        "Virginia Beach",],
    "Washington": [
        "Olympia",
        "Seattle",],
    "West Virginia": [
        "Charleston",
        "Huntington",],
    "Wisconsin": [
        "Madison",
        "Milwaukee",],
    "Wyoming": [
        "Casper",
        "Cheyenne",],
}

In [4]:
years = list(range(2010, 2016+1))

In [5]:
missing_places = { # places missing from citydata
    "Hawaii": [
        "Honolulu",
        "Kauai",
        "Maui",],
    "Nevada": [
        "Carson City",],
    "Tennessee": [
        "Nashville",],
    "Utah": [
        "West Valley City",],
}

In [6]:
corrections = { # alternative names in fbi
    "Charlotte": "charlotte-mecklenburg",
    "Las Vegas": "las vegas metropolitan police department",
    "Savannah": "savannah-chatham metropolitan",
    "West Valley City": "west valley",
    "New York City": "new york"
}

In [7]:
citydata = pd.read_pickle("citydata_crime_data.pkl")

In [8]:
fbi = pd.read_pickle("fbi_crime_data.pkl")

# Processing

In [9]:
data = citydata.copy()

In [10]:
# add populations
data['population'] = np.nan
for state, city, year in zip(data.index.get_level_values('state'),
                             data.index.get_level_values('city'),
                             data.index.get_level_values('year')):
    try:
        fbi.loc[state.lower(), (corrections[city] if city in corrections else city).lower(), int(year)]
    except KeyError:
        continue
    data.loc[state, city, year].loc['population'] = \
            fbi.loc[state.lower(),
                    (corrections[city] if city in corrections else city).lower(),
                    int(year)] \
                   .loc['population']

In [11]:
# add missing cities
for state, cities in missing_places.items():
    for city in cities:
        if city in corrections:
            city = corrections[city]
        try:
            fbi.loc[state.lower(), city.lower()]
        except KeyError:
            print("fbi does not contain {city}, {state}"
                  .format(city=city, state=state))
            continue

fbi does not contain Kauai, Hawaii
fbi does not contain Maui, Hawaii
fbi does not contain Carson City, Nevada


In [12]:
data.to_pickle("crime_data.pkl")

In [13]:
with pd.option_context('display.max_rows', None):
    display(data)

arson  assault  burglary  murder  \
state          city             year                                      
Alabama        Birmingham       2002   211.0   1697.0    4389.0    65.0   
                                2003   175.0   1706.0    4831.0    85.0   
                                2004   142.0   1593.0    5156.0    59.0   
                                2005   136.0   1675.0    4933.0   104.0   
                                2006   228.0   1422.0    4813.0   104.0   
                                2007   221.0   1396.0    4864.0    86.0   
                                2008   134.0   1456.0    5153.0    82.0   
                                2009   135.0   1399.0    5019.0    65.0   
                                2011   123.0   1916.0    5806.0    54.0   
                                2012   117.0   2035.0    4704.0    67.0   
                                2013     NaN   1642.0    4018.0    63.0   
                                2014   145.0   2084.0    3750.0    52.0   
                                2015   145.0   2355.0    3146.0    79.0   
               Montgomery       2002     NaN    629.0    3812.0    30.0   
                                2003     NaN    516.0    4092.0    18.0   
                                2004     NaN    574.0    3734.0    25.0   
                                2005    41.0    656.0    3528.0    29.0   
                                2006     NaN    397.0    3380.0    27.0   
                                2007    53.0    405.0    3317.0    46.0   
                                2008     NaN    348.0    3513.0    23.0   
                                2009     NaN    302.0    3092.0    31.0   
                                2010    50.0    316.0    3279.0    25.0   
                                2011    31.0    284.0    2885.0    31.0   
                                2012    64.0    283.0    3409.0    44.0   
                                2014     NaN    526.0    2548.0    35.0   
                                2015     NaN    578.0    2342.0    33.0   
                                2016     NaN    675.0    2293.0    30.0   
Alaska         Anchorage        2002   141.0   1067.0    1521.0    18.0   
                                2003   128.0   1143.0    1418.0    17.0   
                                2004    82.0   1555.0    1523.0    15.0   
                                2005    76.0   1407.0    1783.0    16.0   
                                2006   106.0   1862.0    1733.0    17.0   
                                2007   145.0   1673.0    1454.0    22.0   
                                2008   106.0   1830.0    1191.0    10.0   
                                2009   111.0   1658.0    1613.0    14.0   
                                2010    86.0   1701.0    1223.0    13.0   
                                2011   126.0   1628.0    1080.0    12.0   
                                2012    97.0   1673.0    1158.0    15.0   
                                2013    74.0   1491.0    1318.0    14.0   
                                2014    81.0   1705.0    1375.0    12.0   
                                2015   106.0   2062.0    1685.0    26.0   
                                2016    69.0   2157.0    2008.0    28.0   
               Juneau           2003     2.0     56.0      99.0     0.0   
                                2004     9.0     99.0     163.0     0.0   
                                2005     5.0     91.0     125.0     0.0   
                                2006     4.0     80.0     194.0     2.0   
                                2007    10.0     86.0     174.0     1.0   
                                2008     5.0     86.0     141.0     0.0   
                                2009     5.0     98.0     266.0     0.0   
                                2010     6.0     89.0     181.0     1.0   
                                2011    18.0    120.0     111.0     0.0   
                                2012     9.0     90.0      94.0 